In [2]:
import pandas as pd
import numpy as np
import vector
import os
from xgboost import XGBClassifier

import HH4b.utils as utils
from HH4b.utils import ShapeVar
import HH4b.plotting as plotting
import HH4b.postprocessing as postprocessing
from HH4b.postprocessing import Region

import hist
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))

from HH4b.postprocessing import load_columns_legacy
from HH4b.postprocessing.PostProcess import add_bdt_scores

In [3]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [19]:
samples = {
    "2022EE": {
        "ttbar": [
            "TTto4Q",
            "TTtoLNu2Q",
            "TTto2L2Nu",
        ],
        "data": [
            "JetMET_Run2022E",
            "JetMET_Run2022F",
            "JetMET_Run2022G",
        ],
        "qcd": [
            "QCD_HT-200to400",
            "QCD_HT-400to600",
            "QCD_HT-600to800",
            "QCD_HT-800to1000",
            "QCD_HT-1000to1200",
            "QCD_HT-1200to1500",
            "QCD_HT-1500to2000",
            "QCD_HT-2000",
        ],
        "diboson": [
            "WW",
            "ZZ",
            "WZ",
        ],
        "vjets": [
            "Wto2Q-3Jets_HT-200to400",
            "Wto2Q-3Jets_HT-400to600",
            "Wto2Q-3Jets_HT-600to800",
            "Wto2Q-3Jets_HT-800",
            "Zto2Q-4Jets_HT-200to400",
            "Zto2Q-4Jets_HT-400to600",
            "Zto2Q-4Jets_HT-600to800",
            "Zto2Q-4Jets_HT-800",
        ],
        "vjetslnu": [
            "WtoLNu-2Jets",
        ],
    },
}

MAIN_DIR = "../../../"
dir_name = "24May1_v12_private_had-tt"
path_to_dir = f"{MAIN_DIR}/../data/skimmer/{dir_name}/"
path_to_dir = f"/eos/uscms/store/user/cmantill/bbbb/skimmer/{dir_name}/"
dirs = {path_to_dir: samples}

load_columns = [
    ("weight", 1),
    ("MET_pt", 1),
    ("nFatJets", 1),
    ("bbFatJetPt", 2),
    ("bbFatJetEta", 2),
    ("bbFatJetPhi", 2),
    ("bbFatJetMsd", 2),
    ("bbFatJetPNetTXbbLegacy", 2),
    ("bbFatJetPNetPXbbLegacy", 2),
    ("bbFatJetPNetPQCDbLegacy", 2),
    ("bbFatJetPNetPQCDbbLegacy", 2),
    ("bbFatJetPNetPQCDothersLegacy", 2),
    ("bbFatJetPNetMassLegacy", 2),
    ("bbFatJetPNetTXbb", 2),
    ("bbFatJetPNetMass", 2),
    ("bbFatJetTau3OverTau2", 2),
    ("bbFatJetPNetQCD0HF", 2),
    ("bbFatJetPNetQCD1HF", 2),
    ("bbFatJetPNetQCD2HF", 2),
]

events_dict = {}
years = ["2022EE"]
for year in years:
    events_dict[year] = {}
    for input_dir, samples in dirs.items():
        events_dict[year] = {
            **events_dict[year],
            **utils.load_samples(
                input_dir,
                samples[year],
                year,
                filters=None,
                columns=utils.format_columns(
                    load_columns
                    + [
                        ("AK8PFJet230_SoftDropMass40", 1),
                        ("AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35", 1),
                    ]
                ),
                reorder_txbb=False,
                variations=False,
            ),
        }

Loaded TTto2L2Nu                                         : 33479 entries
Loaded TTto4Q                                            : 1277910 entries
Loaded TTtoLNu2Q                                         : 1210884 entries
Loaded JetMET_Run2022E                                   : 42773 entries
Loaded JetMET_Run2022F                                   : 193381 entries
Loaded JetMET_Run2022G                                   : 24426 entries
Loaded QCD_HT-1000to1200                                 : 54569 entries
Loaded QCD_HT-1200to1500                                 : 80584 entries
Loaded QCD_HT-1500to2000                                 : 94970 entries
Loaded QCD_HT-2000                                       : 138514 entries


/uscms_data/d3/cmantill/hh/HH4b/src/HH4b/utils.py:346: UserWarning: No events for QCD_HT-200to400!
  warnings.warn(f"No events for {sample}!", stacklevel=1)


Loaded QCD_HT-400to600                                   : 66 entries
Loaded QCD_HT-600to800                                   : 6526 entries
Loaded QCD_HT-800to1000                                  : 30702 entries
Loaded WW                                                : 1460 entries
Loaded WZ                                                : 1807 entries
Loaded ZZ                                                : 524 entries
Loaded Wto2Q-3Jets_HT-600to800                           : 155 entries


/uscms_data/d3/cmantill/hh/HH4b/src/HH4b/utils.py:346: UserWarning: No events for Wto2Q-3Jets_HT-200to400!
  warnings.warn(f"No events for {sample}!", stacklevel=1)
/uscms_data/d3/cmantill/hh/HH4b/src/HH4b/utils.py:346: UserWarning: No events for Wto2Q-3Jets_HT-400to600!
  warnings.warn(f"No events for {sample}!", stacklevel=1)


Loaded Wto2Q-3Jets_HT-800                                : 24993 entries
Loaded Zto2Q-4Jets_HT-400to600                           : 12 entries


/uscms_data/d3/cmantill/hh/HH4b/src/HH4b/utils.py:346: UserWarning: No events for Zto2Q-4Jets_HT-200to400!
  warnings.warn(f"No events for {sample}!", stacklevel=1)


Loaded Zto2Q-4Jets_HT-600to800                           : 2482 entries
Loaded Zto2Q-4Jets_HT-800                                : 21236 entries


In [5]:
list(events_dict["2022EE"]["ttbar"].columns)

[('weight', 0),
 ('MET_pt', 0),
 ('nFatJets', 0),
 ('bbFatJetPt', 0),
 ('bbFatJetPt', 1),
 ('bbFatJetEta', 0),
 ('bbFatJetEta', 1),
 ('bbFatJetPhi', 0),
 ('bbFatJetPhi', 1),
 ('bbFatJetMsd', 0),
 ('bbFatJetMsd', 1),
 ('bbFatJetPNetTXbbLegacy', 0),
 ('bbFatJetPNetTXbbLegacy', 1),
 ('bbFatJetPNetPXbbLegacy', 0),
 ('bbFatJetPNetPXbbLegacy', 1),
 ('bbFatJetPNetPQCDbLegacy', 0),
 ('bbFatJetPNetPQCDbLegacy', 1),
 ('bbFatJetPNetPQCDbbLegacy', 0),
 ('bbFatJetPNetPQCDbbLegacy', 1),
 ('bbFatJetPNetPQCDothersLegacy', 0),
 ('bbFatJetPNetPQCDothersLegacy', 1),
 ('bbFatJetPNetMassLegacy', 0),
 ('bbFatJetPNetMassLegacy', 1),
 ('bbFatJetPNetTXbb', 0),
 ('bbFatJetPNetTXbb', 1),
 ('bbFatJetPNetMass', 0),
 ('bbFatJetPNetMass', 1),
 ('bbFatJetTau3OverTau2', 0),
 ('bbFatJetTau3OverTau2', 1),
 ('bbFatJetPNetQCD0HF', 0),
 ('bbFatJetPNetQCD0HF', 1),
 ('bbFatJetPNetQCD1HF', 0),
 ('bbFatJetPNetQCD1HF', 1),
 ('bbFatJetPNetQCD2HF', 0),
 ('bbFatJetPNetQCD2HF', 1),
 ('AK8PFJet230_SoftDropMass40', 0),
 ('AK8PFJet250

In [20]:
import importlib
import sys

sys.path.append("../boosted/bdt_trainings_run3/")
bdt_model = XGBClassifier()
model_name = "24Apr21_legacy_vbf_vars"
bdt_config = "24Apr21_legacy_vbf_vars"
# model_name = "24May1_legacy_pt300xbb08msd140_2022EE2023_all"
# model_name = "24Apr20_legacy_fix"
# bdt_config = "24Apr20_legacy_fix"
bdt_model.load_model(fname=f"../boosted/bdt_trainings_run3/{model_name}/trained_bdt.model")
# get function
make_bdt_dataframe = importlib.import_module(
    f".{bdt_config}", package="HH4b.boosted.bdt_trainings_run3"
)

legacy_label = "Legacy"

hlts = ["AK8PFJet230_SoftDropMass40"]

events_dict_bdt = {}
for year in years:
    events_dict_bdt[year] = {}
    for key in events_dict[year].keys():
        events = events_dict[year][key]

        bdt_events = make_bdt_dataframe.bdt_dataframe(events)
        preds = bdt_model.predict_proba(bdt_events)
        add_bdt_scores(bdt_events, preds)

        # add variables
        bdt_events["weight"] = events["finalWeight"].to_numpy()

        bdt_events["bdt_score_finebin"] = bdt_events["bdt_score"].copy()

        bdt_events["H1Pt"] = events["bbFatJetPt"].to_numpy()[:, 0]
        bdt_events["H2Pt"] = events["bbFatJetPt"].to_numpy()[:, 1]
        bdt_events["H1Msd"] = events["bbFatJetMsd"].to_numpy()[:, 0]
        bdt_events["H2Msd"] = events["bbFatJetMsd"].to_numpy()[:, 1]
        bdt_events["H1TXbb"] = events[f"bbFatJetPNetTXbb{legacy_label}"].to_numpy()[:, 0]
        bdt_events["H2TXbb"] = events[f"bbFatJetPNetTXbb{legacy_label}"].to_numpy()[:, 1]
        bdt_events["H1PNetMass"] = events[f"bbFatJetPNetMass{legacy_label}"].to_numpy()[:, 0]
        bdt_events["H2PNetMass"] = events[f"bbFatJetPNetMass{legacy_label}"].to_numpy()[:, 1]
        bdt_events["H1TXbbNoLeg"] = events["bbFatJetPNetTXbb"].to_numpy()[:, 0]
        bdt_events["H2TXbbNoLeg"] = events["bbFatJetPNetTXbb"].to_numpy()[:, 1]

        h1 = vector.array(
            {
                "pt": events["bbFatJetPt"].to_numpy()[:, 0],
                "phi": events["bbFatJetPhi"].to_numpy()[:, 0],
                "eta": events["bbFatJetEta"].to_numpy()[:, 0],
                "M": events["bbFatJetPNetMass"].to_numpy()[:, 0],
            }
        )
        h2 = vector.array(
            {
                "pt": events["bbFatJetPt"].to_numpy()[:, 1],
                "phi": events["bbFatJetPhi"].to_numpy()[:, 1],
                "eta": events["bbFatJetEta"].to_numpy()[:, 1],
                "M": events["bbFatJetPNetMass"].to_numpy()[:, 1],
            }
        )
        hh = h1 + h2
        bdt_events["HHPt"] = hh.pt

        bdt_events["hlt"] = np.any(
            np.array([events[trigger].to_numpy()[:, 0] for trigger in hlts if trigger in events]),
            axis=0,
        )
        events_dict_bdt[year][key] = bdt_events

KeyError: ('VBFJetPt', 0)

In [ ]:
control_plot_vars = [
    ShapeVar(
        var="HHPt",
        label=r"$p_{T}^{1,2}$ (GeV)",
        bins=[0, 50, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
        reg=False,
    ),
    ShapeVar(var="H1Msd", label=r"$m_{SD}^{1}$ (GeV)", bins=[30, 30, 300]),
    ShapeVar(var="H2Msd", label=r"$m_{SD}^{2}$ (GeV)", bins=[30, 30, 300]),
    ShapeVar(
        var="H1PNetMass", label=r"$m_{reg}^{1}$ (GeV) " + f"{legacy_label}", bins=[30, 0, 300]
    ),
    ShapeVar(
        var="H2PNetMass", label=r"$m_{reg}^{2}$ (GeV) " + f"{legacy_label}", bins=[30, 0, 300]
    ),
    ShapeVar(var="bdt_score", label=r"BDT score", bins=[20, 0, 1]),
    ShapeVar(
        var="bdt_score_finebin",
        label=r"BDT score",
        bins=[0, 0.005, 0.01, 0.3, 0.68, 0.94, 1],
        reg=False,
    ),
    ShapeVar(var="H1TXbb", label=r"Xbb$^{1}$ " + f"{legacy_label}", bins=[30, 0.0, 1]),
    ShapeVar(var="H2TXbb", label=r"Xbb$^{2}$ " + f"{legacy_label}", bins=[30, 0.0, 1]),
    ShapeVar(var="H1T32", label=r"$\tau_{32}^{1}$", bins=[30, 0, 1]),
    ShapeVar(var="H2T32", label=r"$\tau_{32}^{2}$", bins=[30, 0, 1]),
    ShapeVar(var="H2Pt", label=r"$p_{T}^{2}$ (GeV)", bins=[30, 0, 600]),
    ShapeVar(var="H1Pt", label=r"$p_{T}^{1}$ (GeV)", bins=[30, 0, 600]),
]

In [18]:
events_plot = {}
for year in ["2022EE"]:
    events_plot[year] = {}
    for key in events_dict_bdt[year].keys():
        events = events_dict_bdt[year][key]
        events = events[
            (events["H1TXbb"] > 0.9)
            & (events["H1PNetMass"] > 150)
            & (events["H1PNetMass"] < 200)
            & (events["H1Msd"] > 40)
            & (events["H2PNetMass"] > 50)
            & (events["H2PNetMass"] < 250)
            & (events["H1Pt"] > 300)
            & (events["H2Pt"] > 300)
            # & (events["hlt"])
        ]
        events_plot[year][key] = events

print(events_plot["2022EE"]["ttbar"].columns)

tag = "../../../plots/TTbar/tt-had-xbb09"
os.system(f"mkdir -p {tag}")

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        print(shape_var)
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}",
            sortyield=True,
            show=False,
            log=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            reweight_qcd=True,
            bg_err_mcstat=True,
        )

KeyError: 'ttbar'

# TT region (2)
- Two jets
- 1 tau32?

In [9]:
events_plot["2022EE"]["ttbar"]["H1TXbb"]

5          0.976059
12         0.914985
13         0.973877
19         0.950475
23         0.981077
             ...   
1210850    0.941788
1210861    0.930714
1210863    0.994234
1210864    0.964728
1210877    0.953834
Name: H1TXbb, Length: 450855, dtype: float64

In [16]:
events_plot = {}
for year in ["2022EE"]:
    events_plot[year] = {}
    for key in events_dict_bdt[year].keys():
        events = events_dict_bdt[year][key]
        events = events[
            (events["H1TXbb"] > 0.1)
            & (events["H2TXbb"] > 0.1)
            & (events["H1Msd"] > 40)
            & (events["H1Pt"] > 300)
            & (events["H2Pt"] > 300)
            & (events["H1T32"] < 0.46)
            & (events["H1PNetMass"] > 150)
            & (events["H1PNetMass"] < 200)
            & (events["H2PNetMass"] > 50)
            & (events["H2PNetMass"] < 250)
        ]
        events_plot[year][key] = events

tag = "../../../plots/TTbar/tt-had"
# tag = "tt-had-t32"
os.system(f"mkdir -p {tag}")

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson"]  # , "vjetslnu"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}",
            sortyield=True,
            show=False,
            log=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
        )

In [11]:
samples = {
    "2022EE": {
        "ttbar": [
            "TTto4Q",
            "TTtoLNu2Q",
            "TTto2L2Nu",
        ],
        "data": [
            "EGamma_Run2022E",
            "EGamma_Run2022F",
            "EGamma_Run2022G",
            "Muon_Run2022E",
            "Muon_Run2022F",
            "Muon_Run2022G",
        ],
        "qcd": [
            "QCD_HT-200to400",
            "QCD_HT-400to600",
            "QCD_HT-600to800",
            "QCD_HT-800to1000",
            "QCD_HT-1000to1200",
            "QCD_HT-1200to1500",
            "QCD_HT-1500to2000",
            "QCD_HT-2000",
        ],
        "diboson": [
            "WW",
            "ZZ",
            "WZ",
        ],
        "vjets": [
            "Wto2Q-3Jets_HT-200to400",
            "Wto2Q-3Jets_HT-400to600",
            "Wto2Q-3Jets_HT-600to800",
            "Wto2Q-3Jets_HT-800",
            "Zto2Q-4Jets_HT-200to400",
            "Zto2Q-4Jets_HT-400to600",
            "Zto2Q-4Jets_HT-600to800",
            "Zto2Q-4Jets_HT-800",
        ],
        "vjetslnu": [
            "WtoLNu-4Jets",
        ],
    },
}

MAIN_DIR = "../../../"
dir_name = "24Mar01_v12_semilep-tt"
path_to_dir = f"{MAIN_DIR}/../data/skimmer/{dir_name}/"
dirs = {path_to_dir: samples}

load_columns = [
    ("weight", 1),
    ("MET_pt", 1),
    ("nFatJets", 1),
    ("bbFatJetPt", 2),
    ("bbFatJetEta", 2),
    ("bbFatJetPhi", 2),
    ("bbFatJetMsd", 2),
    ("bbFatJetPNetMass", 2),
    ("bbFatJetPNetXbb", 2),
    ("bbFatJetTau3OverTau2", 2),
    ("ak8FatJetPt", 2),
    ("ak8FatJetEta", 2),
    ("ak8FatJetPhi", 2),
    ("ak8FatJetMsd", 2),
    ("ak8FatJetPNetMass", 2),
    ("ak8FatJetPNetXbb", 2),
    ("ak8FatJetTau3OverTau2", 2),
    # ("bbFatJetPNetQCD0HF", 2),
    # ("bbFatJetPNetQCD1HF", 2),
    # ("bbFatJetPNetQCD2HF", 2),
]

pt_cut = 300
pt_veto = 200
msd_cut = 50
filters = [
    [
        ("('ak8FatJetPt', '0')", ">=", pt_cut),
        ("('ak8FatJetPt', '1')", "<", pt_veto),
        ("('ak8FatJetMsd', '0')", ">=", msd_cut),
        ("('ak8FatJetMsd', '1')", "<", msd_cut),
    ],
]


events_dict = {}
years = ["2022EE"]
for year in years:
    events_dict[year] = {}
    for input_dir, samples in dirs.items():
        events_dict[year] = {
            **events_dict[year],
            **utils.load_samples(
                input_dir,
                samples[year],
                year,
                filters=None,
                columns=utils.format_columns(
                    load_columns
                    + [
                        ("AK8PFJet230_SoftDropMass40", 1),
                        ("AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35", 1),
                    ]
                ),
                reorder_txbb=False,
                variations=False,
            ),
        }

TypeError: load_samples() got an unexpected keyword argument 'columns_mc'

In [ ]:
events_dict_tt = {}
for year in years:
    events_dict_tt[year] = {}
    for key in events_dict[year].keys():
        events = events_dict[year][key]
        events_dict_tt[year][key] = pd.DataFrame()
        events_dict_tt[year][key]["weight"] = events["weight"].to_numpy()[:, 0]

        jet_str = "bb"
        # jet_str = "ak8"
        events_dict_tt[year][key]["H1Msd"] = events[f"{jet_str}FatJetMsd"].to_numpy()[:, 0]
        events_dict_tt[year][key]["H2Msd"] = events[f"{jet_str}FatJetMsd"].to_numpy()[:, 1]
        events_dict_tt[year][key]["H1Pt"] = events[f"{jet_str}FatJetPt"].to_numpy()[:, 0]
        events_dict_tt[year][key]["H2Pt"] = events[f"{jet_str}FatJetPt"].to_numpy()[:, 1]
        events_dict_tt[year][key]["H1T32"] = events[f"{jet_str}FatJetTau3OverTau2"].to_numpy()[:, 0]
        events_dict_tt[year][key]["H2T32"] = events[f"{jet_str}FatJetTau3OverTau2"].to_numpy()[:, 1]
        events_dict_tt[year][key]["H1Xbb"] = events[f"{jet_str}FatJetPNetXbb"].to_numpy()[:, 0]
        events_dict_tt[year][key]["H2Xbb"] = events[f"{jet_str}FatJetPNetXbb"].to_numpy()[:, 1]

        h1 = vector.array(
            {
                "pt": events[f"{jet_str}FatJetPt"].to_numpy()[:, 0],
                "phi": events[f"{jet_str}FatJetPhi"].to_numpy()[:, 0],
                "eta": events[f"{jet_str}FatJetEta"].to_numpy()[:, 0],
                "M": events[f"{jet_str}FatJetPNetMass"].to_numpy()[:, 0],
            }
        )
        h2 = vector.array(
            {
                "pt": events[f"{jet_str}FatJetPt"].to_numpy()[:, 1],
                "phi": events[f"{jet_str}FatJetPhi"].to_numpy()[:, 1],
                "eta": events[f"{jet_str}FatJetEta"].to_numpy()[:, 1],
                "M": events[f"{jet_str}FatJetPNetMass"].to_numpy()[:, 1],
            }
        )
        hh = h1 + h2
        events_dict_tt[year][key]["HHPt"] = hh.pt

In [ ]:
events_dict_tt["2022EE"]["vjetslnu"]

In [ ]:
control_plot_vars = [
    ShapeVar(var="H2Msd", label=r"$m_{SD}^{2}$ (GeV)", bins=[30, 30, 300]),
    ShapeVar(var="H1Msd", label=r"$m_{SD}^{1}$ (GeV)", bins=[30, 30, 300]),
    # ShapeVar(var="H1Mass", label=r"$m_{reg}^{1}$ (GeV)", bins=[30, 0, 300]),
    # ShapeVar(var="H2PNetMass", label=r"$m_{reg}^{2}$ (GeV)", bins=[30, 0, 300]),
    ShapeVar(var="H1Xbb", label=r"Xbb$^{1}$", bins=[30, 0.0, 1]),
    ShapeVar(var="H2Xbb", label=r"Xbb$^{2}$", bins=[30, 0.0, 1]),
    ShapeVar(var="H1T32", label=r"$\tau_{32}^{1}$", bins=[30, 0, 1]),
    ShapeVar(var="H2T32", label=r"$\tau_{32}^{2}$", bins=[30, 0, 1]),
    ShapeVar(var="H2Pt", label=r"$p_{T}^{2}$ (GeV)", bins=[30, 0, 600]),
    ShapeVar(var="H1Pt", label=r"$p_{T}^{1}$ (GeV)", bins=[30, 0, 600]),
    ShapeVar(
        var="HHPt",
        label=r"$p_{T}^{1,2}$ (GeV)",
        bins=[0, 50, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
        reg=False,
    ),
]

ylims = {
    "2022": 5e4,
    "2022EE": 4e5,
    "2023-pre-BPix": 4e5,
}

events_plot = {}
for year in ["2022EE"]:
    events_plot[year] = {}
    for key in events_dict_tt[year].keys():
        events = events_dict_tt[year][key]
        # events = events[(events["H1Msd"] > 140)]
        # events = events[(events["H1Msd"] > 140) & (events["H1T32"]<0.46)]
        events_plot[year][key] = events

print(events_plot["2022EE"]["ttbar"].columns)

tag = "tt-semi-lep"
# tag = "tt-semi-lep-msd140"
# tag = "tt-semi-lep-msd140-t32"
# tag = "tt-semi-lep-bypt"
os.system(f"mkdir -p {tag}")

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson", "vjetslnu"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}",
            sortyield=True,
            show=False,
            log=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            ylim=ylims[year],
        )

control_plot_vars = [
    ShapeVar(
        var="H1Xbb",
        label=r"Xbb$^{1}$",
        bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.92, 0.94, 0.96, 1],
        reg=False,
    ),
]

for year in ["2022EE"]:
    hists = {}
    for shape_var in control_plot_vars:
        if shape_var.var not in hists:
            hists[shape_var.var] = utils.singleVarHist(
                events_plot[year],
                shape_var,
                weight_key="weight",
            )

        bkgs = ["ttbar", "qcd", "vjets", "diboson", "vjetslnu"]
        sigs = []

        plotting.ratioHistPlot(
            hists[shape_var.var],
            year,
            sigs,
            bkgs,
            name=f"{tag}/{shape_var.var}_logx",
            sortyield=True,
            show=False,
            log=True,
            # logx=True,
            plot_significance=False,
            significance_dir=shape_var.significance_dir,
            ratio_ylims=[0.2, 1.8],
            ylim=ylims[year],
        )